In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import widgets, interactive
import warnings
from os import listdir
from os.path import isfile, join
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')
# mypal = plt.rcParams['axes.prop_cycle'].by_key()['color'] # Grab the color pal
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
my_colors = ['teal','firebrick','burlywood','royalblue','mediumpurple','goldenrod','darkgray','palevioletred']
sns.set_palette(sns.color_palette(my_colors))

#### Get the earliest year of which the data of all companies is captured

In [32]:
#Balance sheet
earliest_year = []
companies_data = [f for f in listdir('data/balance sheet') if isfile(join('data/balance sheet', f))]
for company in companies_data:
    bs_df = pd.ExcelFile(f'data/balance sheet/{company}').parse(0).drop('Type',axis=1)
    earliest_year.append(int(bs_df.columns[1][:4]))
#Display the cumulative sum of number of companies by the earliest_year
pd.Series(earliest_year).value_counts().sort_index().cumsum()

1998      1
1999      2
2000      5
2001     11
2002     16
2003     34
2004     61
2005    124
2006    193
2007    252
2008    301
2009    308
2010    314
2011    325
2012    335
2013    348
2014    371
2015    386
2016    397
2017    402
2018    407
dtype: int64

In [33]:
#Financial ratios
earliest_year = []
companies_data = [f for f in listdir('data/financial ratios') if isfile(join('data/financial ratios', f))]
for company in companies_data:
    fr_df = pd.ExcelFile(f'data/financial ratios/{company}').parse(0).drop('Type',axis=1)
    earliest_year.append(int(fr_df.columns[1][:4]))
#Display the cumulative sum of number of companies by the earliest_year
pd.Series(earliest_year).value_counts().sort_index().cumsum()

1998      1
1999      2
2000      5
2001     11
2002     16
2003     34
2004     61
2005    124
2006    193
2007    252
2008    301
2009    308
2010    314
2011    325
2012    335
2013    348
2014    371
2015    386
2016    397
2017    402
2018    407
dtype: int64

- There are 386 companies that have available dataset from year 2015. 

#### Combine financial indicators and return/volatility of stocks from the year 2015 to 2021

In [315]:
def fi_return_volatility(companies_data,year='2015'):
    #Create a list to store the financial indicators data for all companies
    fi_companies_data = []
    #Create list of selective financial indicators
    fi_features = ['Debt to equity','Dividend yield','EV/EBITDA','Net profit margin','ROA','Liabilities',\
        'Cash and cash equivalents','Net revenue','EBITDA/Net revenue','EBIT margin']
    #Create dictionaries to store the return and price volatility of all companies
    return_companies = {}
    volatility_companies = {}
    
    #Loop through all the companies
    for company in companies_data:
        #Extract return and price volatility from price dataset of company and store it in the dictionaries created above
        price_df = pd.ExcelFile(f'data/price/{company}').parse(0).drop([0,1]).rename(columns={'Attributes':'Date'})
        price_df.set_index('Date',inplace=True)
        try:
            price_extracted = price_df.loc[f'{year}-01-01 00:00:00':f'{year}-12-31 00:00:00']['adjust']
            return_companies[company] = np.round((price_extracted.tail(1).values[0] / price_extracted.head(1).values[0] - 1)*100,2)
            volatility_companies[company] = np.round(price_extracted.std(),2)
        except:
            pass
    
    #Loop through all the companies that have available dataset in the return dictionaries
    for company in list(return_companies.keys()):
        #Extract the name of company
        company_name = company.replace('.xlsx','').strip()
        #Load the dataset from balance sheet and financial ratios of company 
        bs_df = pd.ExcelFile(f'data/balance sheet/{company}').parse(0).drop('Type',axis=1)
        bs_df.columns = ['Ratios'] + [year[:4] for year in bs_df.columns[1:]]
        fr_df = pd.ExcelFile(f'data/financial ratios/{company}').parse(0).drop('Type',axis=1)
        try:
            #Extract data from balance sheet and financial ratios by year
            bs_extracted = bs_df.copy()[['Ratios',year]]
            bs_extracted = bs_extracted.set_index('Ratios').T
            bs_extracted.reset_index(drop=True,inplace=True)
            bs_extracted.columns.name = None
            
            fr_extracted = fr_df.copy()[['Ratios',year]]
            fr_extracted = fr_extracted.set_index('Ratios').T
            fr_extracted.reset_index(drop=True,inplace=True)
            fr_extracted.columns.name = None

            #Merge data from balance sheet and financial ratios into a dataframe
            fi_extracted = pd.merge(bs_extracted,fr_extracted,left_index=True,right_index=True,suffixes=('', '_y'))
            fi_extracted.drop(fi_extracted.filter(regex='_y$').columns, axis=1, inplace=True)
            #Extract the selective financial indicators and create new features
            fi_extracted = fi_extracted[fi_features]
            fi_extracted['Net debt to EBITDA'] = (fi_extracted['Liabilities']-fi_extracted['Cash and cash equivalents'])/(fi_extracted['Net revenue']*(fi_extracted['EBITDA/Net revenue']/100))*100
            fi_extracted['Net debt to EBITDA'] = fi_extracted['Net debt to EBITDA'].apply(lambda x:np.round(x,2))
            fi_extracted.rename(columns={'EBIT margin':'Operating profit margin'},inplace=True)
            fi_extracted.insert(column='Company',value=company_name,loc=0)
            fi_extracted.drop(['Liabilities','Cash and cash equivalents','EBITDA/Net revenue','Net revenue'],axis=1,inplace=True)
            fi_extracted['Return'] = return_companies[company]
            fi_extracted['Volatility'] = volatility_companies[company]

            #Append the completed dataset of financial indicators of company to the list that stores the financial indicators data for all companies 
            fi_companies_data.append(fi_extracted)
        except:
            pass
    
    #Return the dataframe that contains the financial indicators data of all companies in selective year
    return pd.concat(fi_companies_data).reset_index(drop=True)

In [316]:
#Get the financial indicators data of all companies from the year 2015 to the year 2021
df_2015 = fi_return_volatility(companies_data,year='2015')
print('Done')
df_2016 = fi_return_volatility(companies_data,year='2016')
print('Done')
df_2017 = fi_return_volatility(companies_data,year='2017')
print('Done')
df_2018 = fi_return_volatility(companies_data,year='2018')
print('Done')
df_2019 = fi_return_volatility(companies_data,year='2019')
print('Done')
df_2020 = fi_return_volatility(companies_data,year='2020')
print('Done')
df_2021 = fi_return_volatility(companies_data,year='2021')
print('Done')

Done
Done
Done
Done
Done
Done
Done


#### Homogenize the number of companies between years

In [335]:
all_companies_list = [list(df_2015['Company'].values),list(df_2016['Company'].values),list(df_2017['Company'].values),list(df_2018['Company'].values),\
    list(df_2019['Company'].values),list(df_2020['Company'].values),list(df_2021['Company'].values)]
final_companies_list = list(set.intersection(*map(set,all_companies_list)))

In [339]:
df_2015 = df_2015[df_2015['Company'].isin(final_companies_list)].reset_index(drop=True)
df_2016 = df_2016[df_2016['Company'].isin(final_companies_list)].reset_index(drop=True)
df_2017 = df_2017[df_2017['Company'].isin(final_companies_list)].reset_index(drop=True)
df_2018 = df_2018[df_2018['Company'].isin(final_companies_list)].reset_index(drop=True)
df_2019 = df_2019[df_2019['Company'].isin(final_companies_list)].reset_index(drop=True)
df_2020 = df_2020[df_2020['Company'].isin(final_companies_list)].reset_index(drop=True)
df_2021 = df_2021[df_2021['Company'].isin(final_companies_list)].reset_index(drop=True)

#### Save completed dataset

In [356]:
writer = pd.ExcelWriter('data/FinancialIndicator_Return_Volatility.xlsx', engine='xlsxwriter')
df_2015.to_excel(writer,index=False,sheet_name='2015')
df_2016.to_excel(writer,index=False,sheet_name='2016')
df_2017.to_excel(writer,index=False,sheet_name='2017')
df_2018.to_excel(writer,index=False,sheet_name='2018')
df_2019.to_excel(writer,index=False,sheet_name='2019')
df_2020.to_excel(writer,index=False,sheet_name='2020')
df_2021.to_excel(writer,index=False,sheet_name='2021')
writer.save()
writer.close()